# LINEAR REGRESS PARAMS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import scipy
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import statsmodels.formula.api as sm
import copy
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import sklearn
from math import sqrt
from functools import reduce
import numpy as np
#import ee
#import geemap
import geopandas as gpd
import rioxarray as rxr
import rasterstats as rs
from rasterio.plot import show
import rasterio

%matplotlib inline

In [2]:
actual = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/iloilo_rice_data_cleaned_imputed_new.csv')
actual = actual[(actual.param == 'yield') & (actual.ecosystem == 'irri')].reset_index(drop=True)
actual = actual[(actual.year == 2018) ].groupby('muni').mean().reset_index(drop=False)
actual.tail()


,muni,year,value
39,Santa Barbara,2018.0,5.003835
40,Sara,2018.0,3.024052
41,Tigbauan,2018.0,4.522636
42,Tubungan,2018.0,3.544776
43,Zarraga,2018.0,4.185845


# IRRIGATED PLOTS VS PARAMS

In [2]:
param_name = 'pcp'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JAS.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Irrigated Rice']

# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_pcp_JAS.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']

# Rename column headings
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
params

#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)

startYear = 2012
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)
df_params = df_params[(df_params.Date <= endYear) & (df_params.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)

merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                 -0.030
Method:                 Least Squares   F-statistic:                    0.5054
Date:                Sun, 19 Dec 2021   Prob (F-statistic):              0.487
Time:                        19:54:33   Log-Likelihood:                 24.910
No. Observations:                  18   AIC:                            -45.82
Df Residuals:                      16   BIC:                            -44.04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.1512      0.175     12.293      0.0

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [108]:
param_name = 'LST_Day_1km'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JAS.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Irrigated Rice']

# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_LST_Day_1km_JAS.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']

# Rename column headings
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
params

#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)

startYear = 2012
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)
df_params = df_params[(df_params.Date <= endYear) & (df_params.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)

merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     2.533
Date:                Tue, 14 Dec 2021   Prob (F-statistic):              0.131
Time:                        21:55:16   Log-Likelihood:                 25.953
No. Observations:                  18   AIC:                            -47.91
Df Residuals:                      16   BIC:                            -46.12
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.7948      0.930      0.854      

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [109]:
param_name = 'LST_Night_1km'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JAS.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Irrigated Rice']

# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_LST_Night_1km_JAS.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']

# Rename column headings
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
params

#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)

startYear = 2012
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)
df_params = df_params[(df_params.Date <= endYear) & (df_params.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)

merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.059
Method:                 Least Squares   F-statistic:                   0.05000
Date:                Tue, 14 Dec 2021   Prob (F-statistic):              0.826
Time:                        21:55:20   Log-Likelihood:                 24.658
No. Observations:                  18   AIC:                            -45.32
Df Residuals:                      16   BIC:                            -43.54
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         1.9820      1.311      1.512

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [110]:
param_name = 'et'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JAS.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Irrigated Rice']

# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_et_JAS.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']

# Rename column headings
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
params

#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)

startYear = 2012
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)
df_params = df_params[(df_params.Date <= endYear) & (df_params.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)

merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.584
Method:                 Least Squares   F-statistic:                     24.87
Date:                Tue, 14 Dec 2021   Prob (F-statistic):           0.000134
Time:                        21:55:25   Log-Likelihood:                 33.070
No. Observations:                  18   AIC:                            -62.14
Df Residuals:                      16   BIC:                            -60.36
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.5044      0.155      9.714      0.0

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [15]:
param_name = 'pet'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JAS.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Irrigated Rice']

# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JAS.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']

# Rename column headings
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
params

#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)

startYear = 2012
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)
df_params = df_params[(df_params.Date <= endYear) & (df_params.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)

merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.450
Model:                            OLS   Adj. R-squared:                  0.415
Method:                 Least Squares   F-statistic:                     13.08
Date:                Tue, 14 Dec 2021   Prob (F-statistic):            0.00232
Time:                        22:30:23   Log-Likelihood:                 30.006
No. Observations:                  18   AIC:                            -56.01
Df Residuals:                      16   BIC:                            -54.23
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.3542      0.255      5.312      0.0

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# RAINFED PLOTS VS PARAMS

In [51]:
param_name = 'pcp'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JFM.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Rain-fed Rice']


# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']

# Rename column headings
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
params

#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)

startYear = 2012
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)
df_params = df_params[(df_params.Date <= endYear) & (df_params.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)

merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)


                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.160
Method:                 Least Squares   F-statistic:                   0.03641
Date:                Tue, 14 Dec 2021   Prob (F-statistic):              0.855
Time:                        23:56:56   Log-Likelihood:                 4.6008
No. Observations:                   8   AIC:                            -5.202
Df Residuals:                       6   BIC:                            -5.043
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.1741      0.333      6.527      0.0

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [42]:
param_name = 'LST_Day_1km'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JFM.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Rain-fed Rice']


# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']

# Rename column headings
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
params

#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)

startYear = 2012
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)
df_params = df_params[(df_params.Date <= endYear) & (df_params.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)

merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.463
Model:                            OLS   Adj. R-squared:                  0.373
Method:                 Least Squares   F-statistic:                     5.165
Date:                Tue, 14 Dec 2021   Prob (F-statistic):             0.0634
Time:                        23:57:09   Log-Likelihood:                 7.0609
No. Observations:                   8   AIC:                            -10.12
Df Residuals:                       6   BIC:                            -9.963
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       4.9310      1.241      3.973      

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [62]:
param_name = 'LST_Night_1km'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JFM.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Rain-fed Rice']

# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']

# Rename column headings
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
params

#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)

startYear = 2013
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)
df_params = df_params[(df_params.Date <= endYear) & (df_params.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)

merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.522
Model:                            OLS   Adj. R-squared:                  0.510
Method:                 Least Squares   F-statistic:                     44.78
Date:                Tue, 14 Dec 2021   Prob (F-statistic):           4.50e-08
Time:                        23:59:24   Log-Likelihood:                 41.363
No. Observations:                  43   AIC:                            -78.73
Df Residuals:                      41   BIC:                            -75.20
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         6.0941      0.611      9.980

In [58]:
param_name = 'et'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JFM.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Rain-fed Rice']


# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']

# Rename column headings
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
params

#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)

startYear = 2012
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)
df_params = df_params[(df_params.Date <= endYear) & (df_params.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)

merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.396
Model:                            OLS   Adj. R-squared:                  0.382
Method:                 Least Squares   F-statistic:                     26.91
Date:                Tue, 14 Dec 2021   Prob (F-statistic):           6.12e-06
Time:                        23:58:08   Log-Likelihood:                 37.702
No. Observations:                  43   AIC:                            -71.40
Df Residuals:                      41   BIC:                            -67.88
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.2294      0.157      7.852      0.0

In [57]:
param_name = 'pet'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JFM.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Rain-fed Rice']

# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']

# Rename column headings
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
params

#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)

startYear = 2012
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)
df_params = df_params[(df_params.Date <= endYear) & (df_params.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)

merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                   0.01149
Date:                Tue, 14 Dec 2021   Prob (F-statistic):              0.915
Time:                        23:58:04   Log-Likelihood:                 26.858
No. Observations:                  43   AIC:                            -49.72
Df Residuals:                      41   BIC:                            -46.19
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0895      0.486      4.303      0.0

# SLOPE AND ELEVATION

In [33]:
param_name = 'elevation'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JFM.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Irrigated Rice']

# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_ch3_elevation.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params['param']=param_name


#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)
df_params

startYear = 2012
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)


merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
merged

formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.062
Method:                 Least Squares   F-statistic:                 0.0005292
Date:                Tue, 14 Dec 2021   Prob (F-statistic):              0.982
Time:                        22:46:40   Log-Likelihood:                 17.000
No. Observations:                  18   AIC:                            -30.00
Df Residuals:                      16   BIC:                            -28.22
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.9144      0.056     34.489      0.0

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [34]:
param_name = 'slope'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JFM.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Irrigated Rice']

# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_ch3_slope.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params['param']=param_name


#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)
df_params

startYear = 2012
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)


merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
merged

formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.055
Method:                 Least Squares   F-statistic:                    0.1092
Date:                Tue, 14 Dec 2021   Prob (F-statistic):              0.745
Time:                        22:47:17   Log-Likelihood:                 17.061
No. Observations:                  18   AIC:                            -30.12
Df Residuals:                      16   BIC:                            -28.34
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.9295      0.048     40.067      0.0

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# RAINFED

In [47]:
param_name = 'elevation'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JAS.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Rain-fed Rice']

# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_ch3_elevation.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params['param']=param_name


#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)
df_params

startYear = 2012
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)


merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
merged

formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     5.005
Date:                Tue, 14 Dec 2021   Prob (F-statistic):             0.0308
Time:                        23:08:43   Log-Likelihood:                 49.895
No. Observations:                  43   AIC:                            -95.79
Df Residuals:                      41   BIC:                            -92.27
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.2812      0.017    132.220      0.0

In [36]:
param_name = 'slope'

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JAS.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Rain-fed Rice']

# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'

# NDVI SUM Data
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_ch3_slope.gpkg'
params = gpd.read_file(params)
params = params.groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params['param']=param_name


#Melt dataframe
df_points = points.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name="CumulativeNDVI")

df_params = params.melt(id_vars=["Class", "ADM3_EN", "param"], 
                 var_name="Date", 
                 value_name=param_name)
df_params

startYear = 2012
endYear = 2017

df_points = df_points[(df_points.Date <= endYear) & (df_points.Date >= startYear)].groupby('ADM3_EN').mean().reset_index(drop=False)


merged = pd.merge(df_points, df_params, left_on='ADM3_EN', right_on='ADM3_EN')
merged

formula = 'CumulativeNDVI ~ ' + param_name
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:         CumulativeNDVI   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     1.448
Date:                Tue, 14 Dec 2021   Prob (F-statistic):              0.236
Time:                        22:48:41   Log-Likelihood:                 48.165
No. Observations:                  43   AIC:                            -92.33
Df Residuals:                      41   BIC:                            -88.81
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.2920      0.019    121.245      0.0